In [ ]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.optimizers import Adam
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = 'Datasets/Train'
valid_path = 'Datasets/Test'

# add preprocessing layer to the front of VGG
vgg = VGG19(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

# don't train existing weights
for layer in vgg.layers:
  layer.trainable = False


# useful for getting number of classes
folders = glob('Datasets/Train/*')

# our layers - you can add more if you want
x = Flatten()(vgg.output)
# x = Dense(1000, activation='relu')(x)
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

# view the structure of the model
model.summary()

# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer=Adam(learning_rate=0.001),
  metrics=['accuracy']
)

train_datagen = ImageDataGenerator(rescale = 1./255,
                                  shear_range = 0.2,
                                  zoom_range = 0.2,
                                  horizontal_flip = True,
                                  rotation_range=45,
                                  width_shift_range=0.3,
                                  height_shift_range=0.3,
                                  fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('Datasets/Train',
                                                target_size = (224, 224),
                                                batch_size = 80,
                                                class_mode = 'categorical')

class_names = list(training_set.class_indices.keys())

# Specify the file name to save
file_name = "class_names.txt"

# Write the content of the class_names list to the text file
with open(file_name, "w") as file:
    for class_name in class_names:
        file.write(class_name + "\n")

print("Class names have been saved to", file_name)

test_set = test_datagen.flow_from_directory('Datasets/Test',
                                            target_size = (224, 224),
                                            batch_size = 80,
                                            class_mode = 'categorical')

# fit the model
r = model.fit(
  training_set,
  validation_data=test_set,
  epochs=20,
  batch_size=80,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)
# loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('Loss vs Validation_loss')

# accuracies
# Assuming r.history['accuracy'] and r.history['val_accuracy'] are sequences
train_acc = np.array(r.history['accuracy']) 
val_acc = np.array(r.history['val_accuracy']) 

plt.plot(train_acc, label='train acc')
plt.plot(val_acc, label='val acc')
plt.legend()
plt.show()
plt.savefig('Accuracy vs Validation_accuracy')

import tensorflow as tf
from keras.models import load_model

model.save('model.h5')